In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Download Dataset**

In [1]:
!mkdir Dataset

In [2]:
import urllib.request

def download_all_classes():
    # List of all class names in the Quick, Draw! dataset
    all_classes = [
        'airplane','banana', 'basketball', 'book',
        'car', 'cat', 'fish', 'flower', 'guitar',
        'house', 'laptop', 'lion', 'moon', 'mountain', 'pencil',
        'star', 'tree', 'umbrella','sun', 'cloud', 'house', 'lightning'
    ]

    base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

    for c in all_classes:
        cls_url = c.replace('_', '%20')
        path = base + cls_url + '.npy'
        print(f'Downloading {c}...')
        urllib.request.urlretrieve(path, 'Dataset/' + c + '.npy')
        print(f'{c} downloaded.')

download_all_classes()


airplane downloaded.


KeyboardInterrupt: 

# **Import Library**

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# **Load and preprocess the dataset using generators**

In [ ]:
def data_generator(data_dir, batch_size):
    class_names = [os.path.splitext(f)[0] for f in os.listdir(data_dir) if f.endswith('.npy')]
    train_data = []
    val_data = []
    test_data = []

    for class_name in class_names:
        data = np.load(os.path.join(data_dir, class_name + '.npy'))
        labels = np.full(data.shape[0], class_names.index(class_name))
        data = data.reshape((-1, 28, 28, 1))  # Reshape the data to (28, 28, 1)
        class_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
        class_dataset = class_dataset.shuffle(len(data))

        train_size = int(0.7 * len(data))
        val_size = int(0.15 * len(data))
        test_size = len(data) - train_size - val_size

        train_class_data = class_dataset.take(train_size)
        val_class_data = class_dataset.skip(train_size).take(val_size)
        test_class_data = class_dataset.skip(train_size + val_size).take(test_size)

        train_data.append(train_class_data)
        val_data.append(val_class_data)
        test_data.append(test_class_data)

    train_dataset = train_data[0]
    val_dataset = val_data[0]
    test_dataset = test_data[0]

    for train_class_data in train_data[1:]:
        train_dataset = train_dataset.concatenate(train_class_data)

    for val_class_data in val_data[1:]:
        val_dataset = val_dataset.concatenate(val_class_data)

    for test_class_data in test_data[1:]:
        test_dataset = test_dataset.concatenate(test_class_data)

    train_dataset = train_dataset.shuffle(len(train_dataset)).batch(batch_size)
    val_dataset = val_dataset.shuffle(len(val_dataset)).batch(batch_size)
    test_dataset = test_dataset.shuffle(len(test_dataset)).batch(batch_size)

    return train_dataset, val_dataset, test_dataset, class_names

# **Load Data**

In [ ]:
# Load data
data_dir = 'Dataset/'

In [ ]:
# Define batch size
batch_size = 256

# Create training, validation, and test datasets
train_dataset, val_dataset, test_dataset, class_names = data_generator(data_dir, batch_size)

# **Build and train the model**

In [ ]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Number of epochs to train
num_epochs = 10  # Change this value to your desired number of epochs

In [ ]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[reduce_lr, early_stopping])

Epoch 1/30
34128/34128 [==============================] - 2100s 61ms/step - loss: 0.3118 - accuracy: 0.9109 - val_loss: 0.2716 - val_accuracy: 0.9228 - lr: 0.0010
Epoch 2/30
34127/34128 [============================>.] - ETA: 0s - loss: 0.2581 - accuracy: 0.9266

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# **Save the trained model**

In [ ]:
model.save('bestModel.h5')

In [ ]:
from google.colab import files

files.download('bestModel.h5')

In [ ]:
all_classes = [
        'airplane','banana', 'basketball', 'book',
        'car', 'cat', 'fish', 'flower', 'guitar',
        'house', 'key', 'laptop', 'lion', 'moon', 'mountain', 'pencil',
        'star', 'tree', 'umbrella','sun', 'cloud', 'house', 'lightning'
    ]

with open("class.txt", "w") as file:
    for item in all_classes:
        file.write(item + "\n")

In [ ]:
from google.colab import files

files.download('classes_name.txt')

In [ ]:
import shutil
from google.colab import files

# Compress the directory into a zip file
shutil.make_archive('Dataset', 'zip', 'Dataset')

# Download the zip file
files.download('Dataset.zip')